# Hyperparameter Tuning using HyperDrive

In [15]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import pandas as pd
ws = Workspace.get(name="quick-starts-ws-135934")
exp = Experiment(workspace=ws, name="lol_match_predictor")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135934
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-135934


In [16]:
# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster"

# Create a provisioning configuration using the Standard_D2_V2 VM and capping it at 4 nodes.
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
# Create compute cluster.
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [17]:
ws = Workspace.from_config()
experiment_name = 'lol_match_predictor'

experiment=Experiment(ws, experiment_name)

In [18]:
data = pd.read_csv("OraclesElixir_lol_match_data_teams.csv")

# Show raw data before it is processed.
data.head()

,side,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,...,goldspent,minionkills,monsterkills,cspm,golddiffat10,xpdiffat10,csdiffat10,golddiffat15,xpdiffat15,csdiffat15
0,Blue,2220,1,17,11,33,17,11,3.0,0.0,...,62275.0,1049.0,216,34.1892,117.0,-137.0,8.0,-825.0,-1665.0,-12.0
1,Red,2220,0,11,17,25,11,17,0.0,0.0,...,62025.0,1021.0,271,34.9189,-117.0,137.0,-8.0,825.0,1665.0,12.0
2,Blue,2227,0,5,21,16,5,21,1.0,0.0,...,56575.0,1011.0,183,32.1688,-2014.0,-2149.0,-34.0,-5484.0,-4833.0,-56.0
3,Red,2227,1,21,5,41,21,5,2.0,0.0,...,63975.0,976.0,270,33.5698,2014.0,2149.0,34.0,5484.0,4833.0,56.0
4,Blue,1711,1,21,4,34,21,4,4.0,1.0,...,50660.0,806.0,242,36.7504,682.0,615.0,25.0,2566.0,2097.0,53.0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [30]:
#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling( {
        "--C": uniform(0.1,10),
        "--max_iter": choice(20,40,60,80,100,150,200,300,400,600,800)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(".", compute_target=cpu_cluster, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(param_sampling,
                                     estimator=estimator,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)






In [31]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [32]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
#TODO: Save the best model

os.makedirs("outputs", exist_ok=True)

joblib.dump(value=best_run.id, filename="outputs/model.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service